In [ ]:
%matplotlib widget
from collections import defaultdict
import glob
import sys
sys.path.append('/Users/nmiles/PACMan_dist/')


from joblib import dump, load
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
import pandas as pd
import pacman2020
from utils import tokenizer

from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

In [ ]:
train_pacman = pacman2020.PACManTrain(cycles_to_analyze=[24, 25])

In [ ]:
train_pacman.read_training_data(parallel=False)

In [ ]:
train_pacman.fit_model(pman.proposal_data['cycle_25'])

In [ ]:
train_pacman.apply_model(pman.proposal_data['cycle_24'], training=True)

In [ ]:
train_pacman.save_model_results(fout="model_results_cy24.txt", training=True)

In [ ]:
pred = pman.model.predict(pman.proposal_data['cycle_24']['cleaned_text'])
pred_prob = pman.model.predict_proba(pman.proposal_data['cycle_24']['cleaned_text'])

In [ ]:
pred_prob[0]

In [ ]:
pman.proposal_data['cycle_24']['encoded_pred_classification'] = pred

In [ ]:
pman.proposal_data['cycle_24']['pred_classification'] = pman.encoder.inverse_transform(pred)

In [ ]:
pman.model_results

In [ ]:
print(classification_report(pman.model_results['encoded_hand_classification'], 
                            pman.model_results['encoded_model_classification']))

In [ ]:
top_pred = 0
top_two_pred = 0
custom_accuracy_dict = {}
for c in pman.encoder.classes_:
    custom_accuracy_dict[c] = {}
for key in custom_accuracy_dict.keys():
    custom_accuracy_dict[key]['top'] = []
    custom_accuracy_dict[key]['top_two'] = []
    custom_accuracy_dict[key]['misclassified'] = []
for num, row in df.iterrows():
    hand_classification = row['hand_classification']
    top_two = row[row.index.str.contains('prob')].sort_values(ascending=False)[:2]
    categories = list(top_two.index)
    categories = [val.replace('_prob','').replace('_',' ') for val in categories]
    if hand_classification == categories[0]:
        custom_accuracy_dict[hand_classification]['top'].append(1)
        top_pred +=1
        top_two_pred +=1
    elif hand_classification in categories:
        custom_accuracy_dict[hand_classification]['top_two'].append(1)
        top_two_pred += 1
    else:
        custom_accuracy_dict[hand_classification]['misclassified'].append(1)

In [ ]:
top_pred/len(df)

In [ ]:
top_two_pred/len(df)

In [ ]:
pman.save_model()

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split

In [ ]:
scores = cross_val_score(
    pman.model, pman.proposal_data['cycle_24']['cleaned_text'], pman.proposal_data['cycle_24']['encoded_hand_classification'], cv=10, scoring='f1_macro')
scores   

In [ ]:
def combine_proposals(pman):
    df1 = pman.proposal_data['cycle_24']
    df2 = pman.proposal_data['cycle_25']
    df = df1.append(df2)
    return df

In [ ]:
total_dataset = combine_proposals(pman)

In [ ]:
scorestotal = cross_val_score(
    pman.model, total_dataset['cleaned_text'], total_dataset['encoded_hand_classification'], cv=10, scoring='f1_macro')
scorestotal  

In [ ]:
total_dataset = combine_proposals(pman)

In [ ]:
total_dataset.info()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(total_dataset['cleaned_text'], total_dataset['encoded_hand_classification'], test_size=0.15, random_state=42)

In [ ]:
len(y_train)/ len(total_dataset['cleaned_text'])

In [ ]:
train_df = pd.DataFrame()
train_df['cleaned_text'] = X_train
train_df['encoded_hand_classification'] = y_train

In [ ]:
test_df = pd.DataFrame()
test_df['cleaned_text'] = X_test
test_df['encoded_hand_classification'] = y_test

In [ ]:
pman.fit_model(train_df)

In [ ]:
test_df

In [ ]:
pred = pman.model.predict(test_df['cleaned_text'])

In [ ]:
print(classification_report(test_df['encoded_hand_classification'], pred))

In [ ]:
len(pred)

In [ ]:
data_out = defaultdict(list)
for i, row in pman.proposal_data['cycle_24'].iterrows():
    data_out['fname'].append(row['fname'])
    data_out['encoded_pred_classification'].append(row['encoded_pred_classification'])
    data_out['pred_classification'].append(row['pred_classification'])
    data_out['hand_classification'].append(row['hand_classification'])
    data_out['encoded_hand_classification'].append(row['encoded_hand_classification'])
    for j, class_prob in enumerate(pred_prob[i]):
        data_out[f"{pman.encoder.classes_[j].replace(' ','_')}_prob"].append(class_prob)

Testing ~sample_weight~ OneVsRest

In [ ]:
vect = TfidfVectorizer(
                max_features=10000,
                use_idf=True,
                norm='l2',
                ngram_range=(1, 2)
            )

In [ ]:
clf = MultinomialNB(alpha=0.05)

In [ ]:
input_tfidf25 = vect.fit_transform(pman.proposal_data['cycle_25']['cleaned_text'])

In [ ]:
input_tfidf24 = vect.transform(pman.proposal_data['cycle_24']['cleaned_text'])

In [ ]:
train_df =  pman.proposal_data['cycle_25']

In [ ]:
clf.fit(input_tfidf25, train_df['encoded_hand_classification'])

In [ ]:
pred = clf.predict(input_tfidf24)

In [ ]:
sum(pred == pman.proposal_data['cycle_24']['encoded_hand_classification'])/len(pred)

In [ ]:
pred_prob = clf.predict_proba(input_tfidf24)

In [ ]:
pred_prob[0]

In [ ]:
np.sum(pred_prob[1])

In [ ]:
sum(np.exp(clf.feature_log_prob_[2]))